In [3]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
tf.__version__

'2.1.0'

In [32]:
dataset_path = "./pokemons/"

In [33]:
train_dir = os.path.join(dataset_path,"train")
validation_dir = os.path.join(dataset_path,"validation")

In [34]:
train_dir

'./pokemons/train'

## Importamos el modelo de GOOGLE

In [8]:
IMG_SHAPE = (128, 128, 3)

In [9]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

In [10]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

## Congelamos el trainable

In [11]:
base_model.trainable = False

#### "Resumimos" la salida del modelo BASE

In [12]:
base_model.output

<tf.Tensor 'out_relu_1/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [14]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
global_average_layer

<tf.Tensor 'global_average_pooling2d_2/Identity:0' shape=(None, 1280) dtype=float32>

###### a lo nuestro, la capa de Salida

In [21]:
prediction_layer = tf.keras.layers.Dense(units=4, activation='softmax')(global_average_layer)
prediction_layer

<tf.Tensor 'dense_4/Identity:0' shape=(None, 4) dtype=float32>

#### A definir el modelo!

In [44]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [45]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

### ¡A compilar el modelo!

In [51]:
model.compile(optimizer='Adam', 
              loss="categorical_crossentropy",
              metrics=['accuracy'])

#### Y nuestras imagenes?
##### ahora usemos generadores de datos (data generator)

In [52]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [53]:
train_generator = data_gen_train.flow_from_directory(train_dir, 
                                                     target_size=(128,128), 
                                                    batch_size=128,
                                                     class_mode='categorical')

Found 891 images belonging to 4 classes.


In [54]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, 
                                                     target_size=(128,128), 
                                                    batch_size=128,
                                                     class_mode='categorical')

Found 359 images belonging to 4 classes.


## Hora de entrenar el modelo (FIT)

In [65]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

  ...
    to  
  ['...']


/home/doneber/anaconda3/envs/mis_herramientas/lib/python3.7/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


  ...
    to  
  ['...']
Train for 7 steps, validate for 3 steps
Epoch 1/10
7/7 [==============================] - 26s 4s/step - loss: 0.2231 - accuracy: 0.9473 - val_loss: 0.6835 - val_accuracy: 0.7437
Epoch 2/10
7/7 [==============================] - 26s 4s/step - loss: 0.2004 - accuracy: 0.9562 - val_loss: 0.6917 - val_accuracy: 0.7242
Epoch 3/10
7/7 [==============================] - 25s 4s/step - loss: 0.1763 - accuracy: 0.9686 - val_loss: 0.6393 - val_accuracy: 0.7688
Epoch 4/10
7/7 [==============================] - 26s 4s/step - loss: 0.1615 - accuracy: 0.9719 - val_loss: 0.6594 - val_accuracy: 0.7409
Epoch 5/10
7/7 [==============================] - 26s 4s/step - loss: 0.1438 - accuracy: 0.9798 - val_loss: 0.6488 - val_accuracy: 0.7409
Epoch 6/10
7/7 [==============================] - 25s 4s/step - loss: 0.1302 - accuracy: 0.9843 - val_loss: 0.6322 - val_accuracy: 0.7549
Epoch 7/10
7/7 [==============================] - 26s 4s/step - loss: 0.1212 - accuracy: 0.9854 - val_loss:

In [60]:
valid_lose, valid_accuracy = model.evaluate_generator(valid_generator)

  ...
    to  
  ['...']


In [61]:
valid_accuracy

0.73537606

In [62]:
dir='./resultadosPokedex/'

In [63]:
if not os.path.exists(dir):
    os.mkdir(dir)
model.save(os.path.join(dir,"modelo.h5"))
model.save_weights(os.path.join(dir,"pesos.h5"))

In [64]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)